### LECTURE & ECRITURE PANDAS <=> SQL

* énoncé:
  1. télécharger l'url et écrire en local si **ce n'est pas déjà fait**
     > ATTENTION encodage **iso-8859-1**
  2. se donner un dataframe avec le "Nom de domaine" et le "Pays BE" et un millions de ligns
  3. renommer les colonnes du df comme les colonnes de la table domain_name
  4. créer la base de données dns.db depuis le fichier .sql
  5. connecter sqlaclhemy sur la bdd
  6. utiliser Dataframe.to_sql pour écrire dans la table domain_name
  7. lire table dans un dataframe

In [ ]:
import os
import sqlite3
import pandas as pd
from time import time
import numpy as np

# pip install SQLAlchemy
# créer une connexion de bdd
from sqlalchemy import create_engine
# pour créer / modifier un schéma de db
from sqlalchemy.types import Integer, String, Text

URL = "http://www.afnic.fr/wp-media/ftp/documentsOpenData/202105_OPENDATA_A-NomsDeDomaineEnPointFr.zip"